<a href="https://colab.research.google.com/github/awelmisin/faceswap-google-colab/blob/main/Faceswap_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

Faceswap - Google Colab

You also need Faceswap GUI for this to work, since you can't extract faces here, yet.

Files you need to upload on Google Drive
1.   face_a.zip (source face pics you want to swap)
2.   face_b.zip (faces you want to get swapped)

> Zipped files with extracted faces and alignments.fsa

3.   train.ini
> Your train.ini from your local system, in Windows this is in: C:\Users\YourName\faceswap\config

Here is what filesystem looks like in Google Drive
*   /colab_files/faceswap/faces/face_a.zip
*   /colab_files/faceswap/faces/face_b.zip
*   /colab_files/faceswap/config/train.ini

Recommended folders to create on Google Drive:
*   /colab_files/faceswap/models/YourModelName
*   /colab_files/faceswap/output/timelapse

Here is what filesystem should look like in this Google Colab after the "Mount Google Drive" step below.
*   /content/drive/My Drive/colab_files/faceswap/faces/face_a.zip
*   /content/drive/My Drive/colab_files/faceswap/faces/face_b.zip
*   /content/drive/My Drive/colab_files/faceswap/config/train.ini

# Usage Notes

This is my personal SOP for handling the Faceswap process. I find it is best to tweak your settings, extract, test your training, and convert everything using your local system.

But training on my laptop took weeks to get the same results that you can get from just days by using a Google Colab.

*   Extract, Align, then Zip all Faces A
*   Extract, Align, then Zip all Faces B
*   Modify the "Run Training" script below to your preferences
*   Train in this Colab Notebook
*   Watch Google Drive timestamp folder to see how the training is improving
*   Download YourModelName to your local system
*   Convert using the downloaded model


# Keep Session Alive
 (it may not work due to latest Google Colab Captcha changes.)



In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# Check your GPU
If it isn't **Tesla T4** or **Tesla P100**, go Runtime->Factory Reset Runtime until you get one.

In [ ]:
!nvidia-smi

import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

# Setup Faceswap

In [ ]:
#@title Set Time Zone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Turkey /etc/localtime
!date

#above is for HST, you can find yours in
#/usr/share/zoneinfo


Sat May 29 14:19:09 +03 2021


In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Get your training data(aka the faces you extracted in app)
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip" .

!unzip face_a.zip -d face_a
!unzip face_b.zip -d face_b

!rm face_a.zip
!rm face_b.zip


In [ ]:
#@title Get the latest Faceswap
!git clone https://github.com/deepfakes/faceswap.git

In [ ]:
#@title Copy your configuration(train.ini) file
!cp "/content/drive/My Drive/colab_files/faceswap/config/train.ini" faceswap/config/
!ls -lA faceswap/config/
!cat faceswap/config/train.ini

In [ ]:
#@title Install Tensorflow

!pip install -r faceswap/requirements_nvidia.txt

In [ ]:
!python3 faceswap/tools.py mask -a 'face_b/alignments.fsa' -i 'face_b' -it faces -M vgg-clear -p all -o '/drive/MyDrive/colab_files/faceswap/faces/facexb/mask' -b 3 -t 4 -ot mask -L INFO


# Run Training
Iteration number is up to you but it's recommended that you should do until the number you are okay with. You should check your timelapse folder for that.

And if my calculations are right, trainer is doing nearly 360 iterations in 10 mins, which means 2160 iterations in 1 hr.

In [ ]:
#set variables start
num_iterations = "100000"
save_every = "360"
save_model_every = "25000"
batch_num = "16"
num_gpus = "1"

trainer_type = "villain"

model_dir = "/content/drive/My Drive/colab_files/faceswap/models/YourModelName"
alignments_file_a = "face_a/2_alignments.fsa"
alignments_file_b = "face_b/alignments.fsa"
timelapse_dir = "/content/drive/My Drive/colab_files/faceswap/output/timelapse"
#set variables end

!python3 faceswap/faceswap.py train \
  -A 'face_a' \
  -B 'face_b'  \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia 'face_a' \
  -tib 'face_b' \
  -to '{timelapse_dir}'


# Convert

In [ ]:
!python3 faceswap/faceswap.py convert -i '/content/drive/My Drive/colab_files/faceswap/Editor.avi' -o '/content/drive/My Drive/colab_files/faceswap/Results' -al '/content/drive/My Drive/colab_files/faceswap/2_alignments.fsa' -m '/content/drive/My Drive/colab_files/faceswap/models/YourModelName' -c match-hist -M none -w ffmpeg -osc 100 -l 0.4 -j 0 -t realface -L INFO